In [ ]:
!ls

In [ ]:
import pandas as pd
import numpy as np

## Create key 'node_ids'

In [ ]:
df_file_1_mapping=pd.read_csv('file_1_mapping_new.csv')
df_file_1_mapping

In [ ]:
df_file_1_mapping.info()

In [ ]:
# replace NaN values with an empty string
df_file_1_mapping=df_file_1_mapping.fillna('')
df_file_1_mapping

In [ ]:
df_file_1_mapping.info()

In [ ]:
# use agg() to concat 'fac_no', 'code_no', 'fac_type', 'fac_name' columns
df_file_1_mapping['unique_key']=df_file_1_mapping[['fac_no', 'code_no', 'fac_type', 'fac_name']].agg('_'.join, axis=1)
df_file_1_mapping

In [ ]:
df_file_1_mapping=df_file_1_mapping.drop(['fac_no', 'code_no', 'fac_type', 'fac_name'], axis=1)
df_file_1_mapping

In [ ]:
# export to csv
# index_label: column label for index column(s) if desired
df_file_1_mapping.to_csv('nodeidx2uniquekey.csv', index_label='node_idx')

In [ ]:
node_ids={}

for i in range(546):
    key=df_file_1_mapping.iloc[[i], [0]].values
    node_ids[key[0][0]]=i

In [ ]:
node_ids

## Create keys '0' to '167'

### Create key 'edges'

In [ ]:
df_file_2_edge=pd.read_csv('file_2_edge_new.csv')
df_file_2_edge

In [ ]:
df_file_2_edge=df_file_2_edge[['fac_no1', 'code_no1', 'fac_type1', 'fac_name1', 'fac_no2', 'code_no2', 'fac_type2', 'fac_name2']]
df_file_2_edge

In [ ]:
df_file_2_edge.info() # 1 null value in 'fac_no1', 3 null values in 'fac_no2'

In [ ]:
# replace NaN values with an empty string
df_file_2_edge=df_file_2_edge.fillna('')
df_file_2_edge

In [ ]:
df_file_2_edge.info()

In [ ]:
# use agg() to concat 'fac_no1', 'code_no1', 'fac_type1', 'fac_name1' columns
df_file_2_edge['unique_key1']=df_file_2_edge[['fac_no1', 'code_no1', 'fac_type1', 'fac_name1']].agg('_'.join, axis=1)
df_file_2_edge

In [ ]:
# use agg() to concat 'fac_no2', 'code_no2', 'fac_type2', 'fac_name2' columns
df_file_2_edge['unique_key2']=df_file_2_edge[['fac_no2', 'code_no2', 'fac_type2', 'fac_name2']].agg('_'.join, axis=1)
df_file_2_edge

In [ ]:
df_file_2_edge=df_file_2_edge[['unique_key1', 'unique_key2']]
df_file_2_edge

In [ ]:
!ls

In [ ]:
df_idx=pd.read_csv('nodeidx2uniquekey.csv')
df_idx

In [ ]:
# find node_idx1 for unique_key1
df_file_2_edge=pd.merge(df_file_2_edge, df_idx, how='left', left_on='unique_key1', right_on='unique_key')
df_file_2_edge

In [ ]:
df_file_2_edge=df_file_2_edge[['unique_key1', 'unique_key2', 'node_idx']]
df_file_2_edge

In [ ]:
df_file_2_edge.info()

In [ ]:
# find node_idx2 for unique_key2
df_file_2_edge=pd.merge(df_file_2_edge, df_idx, how='left', left_on='unique_key2', right_on='unique_key')
df_file_2_edge

In [ ]:
df_file_2_edge.info()

In [ ]:
df_file_2_edge=df_file_2_edge[['node_idx_x', 'node_idx_y']]
df_file_2_edge

In [ ]:
edges=df_file_2_edge.values
edges

In [ ]:
edges.shape

### Create key 'weights'

In [ ]:
!ls

In [ ]:
df_file_2_weight=pd.read_csv('file_2_edge_new.csv')
df_file_2_weight

In [ ]:
df_file_2_weight=df_file_2_weight.drop(['fac_no1', 'code_no1', 'fac_type1', 'fac_name1', 'fac_no2', 'code_no2', 'fac_type2', 'fac_name2'], axis=1)
df_file_2_weight

In [ ]:
# replace NaN values with 0
df_file_2_weight=df_file_2_weight.fillna(0)
df_file_2_weight

df_file_2_weight.info(verbose=True, show_counts=True)

In [ ]:
weights=np.transpose(df_file_2_weight.values)
weights

In [ ]:
weights.shape

In [ ]:
len(weights[0])

### Create key 'y'

In [ ]:
!ls

In [ ]:
df_node_label=pd.read_csv('node-label.csv', header=None)
df_node_label

In [ ]:
label=df_node_label.values
label

In [ ]:
label=label.flatten()
label

In [ ]:
len(label)

In [ ]:
y=[]

for i in range(168): # 168 time periods
    gt=[]
    for j in range(546): # 546 nodes
        gt.append(label[i+j*168])
    y.append(gt)

In [ ]:
y=np.array(y)
y

In [ ]:
y.shape

### Create key 'X'

In [ ]:
!ls

In [ ]:
df_mapping=pd.read_csv('file_1_mapping_new.csv')
df_mapping

In [ ]:
# replace NaN values with an empty string
df_mapping=df_mapping.fillna('')
df_mapping

In [ ]:
df_file_3_feature=pd.read_csv('file_3_feature.csv')
df_file_3_feature

In [ ]:
# replace NaN values with an empty string
df_file_3_feature=df_file_3_feature.fillna('')
df_file_3_feature

In [ ]:
# align to the order of file_1_mapping
# df_file_1_mapping LEFT JOIN df_file_3_feature
df_file_3_feature=pd.merge(df_mapping, df_file_3_feature, how='left', on=['fac_no', 'code_no', 'fac_type', 'fac_name'])
df_file_3_feature

In [ ]:
df_file_3_feature=df_file_3_feature.drop(['fac_no', 'code_no', 'fac_type', 'fac_name'], axis=1)
df_file_3_feature

In [ ]:
X=df_file_3_feature.to_numpy()
X

In [ ]:
X=np.transpose(X)
X

In [ ]:
X.shape

In [ ]:
for i in range(168):
    for j in range(546):
        X[i][j]=X[i][j].split(',')
        
X

In [ ]:
for i in range(168):
    for j in range(546):
        X[i][j]=['0' if e=='' else e for e in X[i][j]]
        X[i][j]=[float(e) for e in X[i][j]]
        
X

In [ ]:
for i in range(168):
    for j in range(546):
        X[i][j]=np.array(X[i][j])
        
X

In [ ]:
X.shape

In [ ]:
# set missing value
X[42][543]=np.array([0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
X[42][543]

In [ ]:
ans=[]

for i in range(168):
    ans.append(np.stack(tuple(X[i][j] for j in range(546)), axis=0))

In [ ]:
ans

In [ ]:
ans=np.array(ans)

In [ ]:
ans.shape

In [ ]:
ans[0]

In [ ]:
np.stack(tuple(X[0][i] for i in range(546)), axis=0)

## Create dictionary

In [ ]:
outer_dict={}

for i in range(168): # 168 time periods
    inner_dict={}
    
    inner_dict['index']=i
    inner_dict['edges']=edges
    inner_dict['weights']=weights[i]
    inner_dict['y']=y[i]
    inner_dict['X']=ans[i]
    
    outer_dict[str(i)]=inner_dict
    
outer_dict

In [ ]:
outer_dict['time_periods']=168
outer_dict['node_ids']=node_ids

outer_dict

In [ ]:
type(outer_dict)

In [ ]:
# convert from Python to JSON
# I found the best solution if you have nested numpy arrays in a dictionary:
import json

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
with open('output.json', 'w', encoding='utf8') as f:
    json.dump(outer_dict, f, ensure_ascii=False, cls=NumpyEncoder)